In [39]:
!pip3 install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 2.3 MB/s eta 0:00:0000:0100:01


In [211]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
raw_data = pd.read_csv('winequality-white.csv', sep='\t', header=None)  # Using tab as separator

#Display the first few rows to inspect the data
#print("Original DataFrame shape:", df.shape)
#print(df.head())

# Split the first column (which contains all attributes) by semicolons
df_split = raw_data[0].str.split(';', expand=True)

# Extract the header from the first row and create a list of attribute names
attribute_names = df_split.iloc[0].str.replace('"', '', regex=False).str.strip()

# Create a DataFrame from the remaining rows (excluding the first one)
df = df_split[1:]

# Assign the header to the DataFrame
df.columns = attribute_names

# Strip quotes from each attribute
#df = df_split.apply(lambda x: x.str.replace('"', ''))
# Convert only the numeric rows to float, using `pd.to_numeric` to handle conversion
df = df.apply(pd.to_numeric, errors='coerce')

print(df)

# Check the resulting DataFrame
print("Split DataFrame shape:", df.shape)

0     fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1               7.0              0.27         0.36            20.7      0.045   
2               6.3              0.30         0.34             1.6      0.049   
3               8.1              0.28         0.40             6.9      0.050   
4               7.2              0.23         0.32             8.5      0.058   
5               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4894            6.2              0.21         0.29             1.6      0.039   
4895            6.6              0.32         0.36             8.0      0.047   
4896            6.5              0.24         0.19             1.2      0.041   
4897            5.5              0.29         0.30             1.1      0.022   
4898            6.0              0.21         0.38             0.8      0.020   

0     free sulfur dioxide  

In [154]:
# Assuming the last column is the target variable
# X is the features, Y is the target
X = df.iloc[:,:-1]   # All columns except the last one
Y = df.iloc[:, -1]   # The last column is the target
#print(Y)
# Check the split data
print("Feature Matrix (X) shape:", X.shape)
print("Target Vector (Y) shape:", Y.shape)

Feature Matrix (X) shape: (4898, 11)
Target Vector (Y) shape: (4898,)


In [255]:
# Define your percentages
train = 0.7  # 70% for training
# validation = 0.15 # 15% for validation
# The remaining percentage for testing will be (100 - t - v)% = 15%

#Define learning rate
learning_rate = 0.00001
epochs = 8000
alpha = 0.1  # Regularization parameter, adjust as needed

# Proceed with splitting the dataset
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size = (1-train), random_state = 1)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state= 1)

# To confirm the data is correctly split
#print(X_train)

# Now you can use X_train, Y_train, etc.
# Batch Gradient Descent without Regularization
W_batch = batch_gradient_descent(X_train, Y_train, learning_rate, epochs)
print("Weights from Batch GD without Regularization:", W_batch)
# We tried 100,000 interations and found that after almost 8000 iterations, MSE was almost stagnant around 0.67.  

Epoch 0: MSE = 36.896698096734035
Epoch 10: MSE = 4.178193567348551
Epoch 20: MSE = 4.151633043031026
Epoch 30: MSE = 4.125471301037569
Epoch 40: MSE = 4.09950829533482
Epoch 50: MSE = 4.073742449328059
Epoch 60: MSE = 4.048172203716284
Epoch 70: MSE = 4.02279601489945
Epoch 80: MSE = 3.997612354610898
Epoch 90: MSE = 3.9726197095689613
Epoch 100: MSE = 3.9478165811465127
Epoch 110: MSE = 3.923201485057292
Epoch 120: MSE = 3.898772951057954
Epoch 130: MSE = 3.8745295226648255
Epoch 140: MSE = 3.8504697568844337
Epoch 150: MSE = 3.826592223956932
Epoch 160: MSE = 3.8028955071116077
Epoch 170: MSE = 3.7793782023337092
Epoch 180: MSE = 3.756038918141872
Epoch 190: MSE = 3.732876275375488
Epoch 200: MSE = 3.7098889069913867
Epoch 210: MSE = 3.6870754578692564
Epoch 220: MSE = 3.6644345846252544
Epoch 230: MSE = 3.6419649554333033
Epoch 240: MSE = 3.6196652498535977
Epoch 250: MSE = 3.597534158667887
Epoch 260: MSE = 3.5755703837211033
Epoch 270: MSE = 3.5537726377689687
Epoch 280: MSE = 3.

In [246]:
# Batch Gradient Descent
def batch_gradient_descent(X, Y, learning_rate, epochs):
    m, n = X.shape
    W = np.random.randn(n + 1) * 0.001  # Weight vector initializatio- we want to keep it very close to a zero vector
    X = np.c_[np.ones((m, 1)), X] # Add bias term
    #print('X_b----', X_b)

    for epoch in range (epochs):
        Y_hat = np.dot(X,W) # Compute predictions
        mse = np.mean((Y - Y_hat) ** 2) # Mean Squared Error
        gradient = (2/m) * np.dot(X.T, (Y_hat - Y)) # Gradient
        W = W-learning_rate * gradient  # Update weights

        if epoch % 10 == 0:
            print(f'Epoch {epoch}: MSE = {mse}')
    return Y_hat

    
    

In [ ]:
git add regression.ipynb